### Installing the required libraries for the forecasting system.

In [ ]:
!pip install yfinance --quiet

### Importing the required module. 

In [ ]:
import pandas as pd
import numpy as np

class StockData :
    def __init__(self, name_of_stock, start_date, end_date, data_type="csv") :
        super(StockData, self).__init__()
        self. name_of_stock = name_of_stock
        self.start_date = start_date
        self.end_date = end_date
        self.data_type = data_type

    def getStockDataToCSV(self, **kwargs) :
        if self.data_type == "csv" :
            stock_data = yf.download(self.name_of_stock, start=self.start_date, end=self.end_date)
            if kwargs["verbose"] > -1 : 
                print("{}The Data is : {}".format("\n", "\n"))
                print(stock_data.head(kwargs["num_samples"]))
            result = stock_data.to_csv(kwargs["file_name"], encoding=kwargs["file_encoding"], index=True)
            return True
        else :
            print("Only CSV file is supported, change the data_type....")
            return False


if __name__ == "__main__" :
    STOCK = "AAPL"
    START = "2018-08-11" # Date in the format of yyyy-mm-dd
    END = "2019-08-11" # Date in the format of yyyy-mm-dd
    FILE_NAME = "{}-{}.csv".format(STOCK, END)
    VERBOSE = -2
    NUM_SAMPLES = 15
    ENCODING = "utf-8"

    stock = StockData(STOCK, START, END)
    result = stock.getStockDataToCSV(num_samples=NUM_SAMPLES, 
                                     verbose=VERBOSE, 
                                     file_name=FILE_NAME, 
                                     file_encoding=ENCODING)
    if result :
        print("Data Generated Successfully....")
    else :
        print("Internal Exception occured try again....")
        print("Try again....")


## Main Code for the Stock Prediction System starts here.

### Dataset accessing with the help of Yahoo Finance API 

In [ ]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf

stock_data = yf.download('AAPL', start='2015-08-11', end='2022-08-11')
stock_data.head()

In [ ]:
result = stock_data.to_csv("APPL-2019.csv", encoding='utf-8', index=True)

In [ ]:
result

In [ ]:
len(stock_data)

In [ ]:
plt.figure(figsize=(10, 5))
plt.title('Stock Prices History for Open Values')
plt.plot(stock_data["Open"])
plt.xlabel('Date')
plt.ylabel('Prices ($)')

plt.figure(figsize=(10, 5))
plt.title('Stock Prices History for Close Values')
plt.plot(stock_data["Close"], color="y")
plt.xlabel('Date')
plt.ylabel('Prices ($)')


### Dataset preprocessing and training and validation tensor generation.

In [ ]:
close_prices = stock_data['Close']
num_previous_days = 150

values = close_prices.values
training_data_len = math.ceil(len(values)* 0.8)

x_train = []
y_train = []
x_test = []
y_test = values[training_data_len:]
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(values.reshape(-1,1))

train_data = scaled_data[0: training_data_len, :]

for i in range(num_previous_days, len(train_data)):
    x_train.append(train_data[i-num_previous_days:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

test_data = scaled_data[training_data_len-num_previous_days: , : ]

for i in range(num_previous_days, len(test_data)):
    x_test.append(test_data[i-num_previous_days:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
print("The shapes of the final training tensors are : {} and {}".format(x_train.shape, y_train.shape))
print("The shapes of the validation tensors are : {} and {}".format(x_test.shape, y_test.shape))

### Creating a class of LSTM Neural Network for the Analysis of the Stocks Market Prices

In [ ]:
import warnings     
warnings.filterwarnings("ignore")


class LSTMNeuralNetwork :
    def __init__(self, num_lstm_layers, lstm_config, learning_rate, optimizer_type) :
        super(LSTMNeuralNetwork, self).__init__()
        self.num_lstm_layers = num_lstm_layers
        self.lstm_config = lstm_config
        self.learning_rate = learning_rate
        self.optimizer_type = optimizer_type

    def CuDnnLSTMNetwork(self, **kwargs) :
        model = tf.keras.models.Sequential()
        model.add(tf.compat.v1.keras.layers.CuDNNLSTM(kwargs["num_cudnn_units"], 
                                                      return_sequences=True, 
                                                      input_shape=(x_train.shape[1], 1)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(rate=kwargs["layer_dropout"]))

        model.add(tf.compat.v1.keras.layers.CuDNNLSTM(units = (kwargs["num_cudnn_units"] + \
                                                               kwargs["internal_cudnn_embedding"])))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(rate=kwargs["layer_dropout"] + kwargs["internal_dropout_embedding"]))
        
        # Multi-Layered Perceptron
        model.add(tf.keras.layers.Dense(kwargs["num_mlp_neurons"]))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.27))
        model.add(tf.keras.layers.Dropout(rate=kwargs["mlp_dropout"]))
        model.add(tf.keras.layers.BatchNormalization())

        model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

        if self.optimizer_type == "adam" :
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate), 
                        loss='mean_squared_error', metrics=["mse"])
            return model
        elif self.optimizer_type == "rmsprop" :
            model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate, 
                                                                momentum=0.45), 
                          loss="mean_squared_error", metrics=["mse"])
            return model
        elif self.optimizer_type == "nadam" :
            model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=self.learning_rate), 
                          loss="mean_squared_error", metrics=["mse"])
            return model

    def CompoundLSTMNetwork(self, **kwargs) :
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.LSTM(kwargs["units"], return_sequences=True, 
                                       input_shape=(x_train.shape[1], 1)))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.27))
        model.add(tf.keras.layers.BatchNormalization())

        model.add(tf.keras.layers.LSTM(kwargs["units"] + kwargs["lstm_embedding"], 
                                       return_sequences=False, 
                                       dropout=kwargs["lstm_dropout"]))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.27))
        model.add(tf.keras.layers.BatchNormalization())

        model.add(tf.keras.layers.Dense(kwargs["num_mlp_neurons"]))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.27))
        model.add(tf.keras.layers.Dropout(rate=kwargs["mlp_dropout"]))
        model.add(tf.keras.layers.BatchNormalization())

        model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

        if self.optimizer_type == "adam" :
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate), 
                        loss='mean_squared_error', metrics=["mse"])
            return model
        elif self.optimizer_type == "rmsprop" :
            model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate, 
                                                                momentum=0.45), 
                          loss="mean_squared_error", metrics=["mse"])
            return model
        elif self.optimizer_type == "nadam" :
            model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=self.learning_rate), 
                          loss="mean_squared_error", metrics=["mse"])
            return model

    def StackedLSTMNetwork(self, **kwargs) :
        init = tf.keras.initializers.glorot_normal(seed=None)
        init1 = tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05)
        input_tensor = x = tf.keras.layers.Input(shape=(x_train.shape[1], 1))
        
        for i in range(self.num_lstm_layers) :
            name = 'layer_lstm_{0}'.format(i+1)
            if ( (i == 0) and (self.num_lstm_layers == 1) ):
                x = tf.keras.layers.LSTM(units=self.lstm_config[0], 
                                         dropout=kwargs["dropout"], 
                                         recurrent_dropout=kwargs["recurrent_dropout"], 
                        return_sequences=False, kernel_initializer=init, 
                        activation=kwargs["activation"], name=name)(x)
            elif (i != (self.num_lstm_layers - 1) ) :
                x = tf.keras.layers.LSTM(units=self.lstm_config[1], 
                                         dropout=kwargs["dropout"], 
                                         recurrent_dropout=kwargs["recurrent_dropout"],  
                        return_sequences=True, kernel_initializer=init, 
                        activation=kwargs["activation"], name=name)(x)
            else:
                x = tf.keras.layers.LSTM(units=self.lstm_config[2], 
                                         dropout=kwargs["dropout"], 
                                         recurrent_dropout=kwargs["recurrent_dropout"],  
                        return_sequences=False, kernel_initializer=init, 
                        activation=kwargs["activation"], name=name)(x)
        
        # Multi-Layered Perceptron Block
        x = tf.keras.layers.Dense(250)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(rate=0.20)(x)
        x = tf.keras.layers.Dense(50)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(rate=0.10)(x)

        x = tf.keras.layers.Dense(1, activation='linear', kernel_initializer= init1)(x)

        model = tf.keras.models.Model(input_tensor, x)
        if self.optimizer_type == "rmsprop" :
            optimizer = tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate, rho=0.9, epsilon=None, decay=0.0, momentum=0.45)
            model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])
            return model
        elif self.optimizer_type == "adam" :
            optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
            model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mse"])
            return model
        elif self.optimizer_type == "nadam" :
            optimizer = tf.keras.optimizers.Nadam(learning_rate=self.learning_rate)
            model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mse"])
            return model
        else :
            optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
            model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["mse"])


if __name__ == "__main__" :

    # Hyper-Parameters for the Neural Network Class
    num_internal_lstm_layers = 2
    network_configuration = [100, 200, 100]
    learningRate = 0.00025
    optimizerType = "adam"
    
    # Training hyper-parameters of the system.
    num_epochs = 50
    batch_size = 128
    validation_split = 0.2

    # Stacked LSTM Hyper-parameters (**kwargs)
    stacked_lstm_dropout = 0.25
    stacked_lstm_recurrent_dropout = 0.25
    stacked_lstm_activation = "sigmoid"

    # Compound LSTM Hyper-Parameters (**kwargs)
    compound_lstm_units = 150
    compound_lstm_embedding = 150
    compound_lstm_dropout = 0.25
    num_mlp_units_compound_lstm = 256
    mlp_dropout_compound_lstm = 0.15

    # CuDnn LSTM Hyper-Parameters (**kwargs)
    cudnn_units = 275
    cudnn_layer_dropout = 0.15
    cudnn_embedding_dim = 150
    cudnn_dropout_embedding = 0.15
    cudnn_mlp_units = 512
    cudnn_mlp_dropout = 0.2


    # Initialization of the LSTM Neural Network Class
    nn = LSTMNeuralNetwork(num_internal_lstm_layers, network_configuration, 
                            learningRate, optimizerType)

    # The First Model
    model_1 = nn.StackedLSTMNetwork(dropout=stacked_lstm_dropout, 
                                        recurrent_dropout=stacked_lstm_recurrent_dropout, 
                                        activation=stacked_lstm_activation)
    print("{}The Stacked LSTM Internal Model Architectural Details Are : {}".format("\n", "\n"))
    print(model_1.summary())

    print("\n")
    # The Second Model
    model_2 = nn.CompoundLSTMNetwork(units=compound_lstm_units, 
                                        lstm_embedding=compound_lstm_embedding, 
                                        lstm_dropout=compound_lstm_dropout, 
                                        num_mlp_neurons=num_mlp_units_compound_lstm, 
                                        mlp_dropout=mlp_dropout_compound_lstm)
    print("{}The Compound LSTM Internal Model Architectural Details Are : {}".format("\n", "\n"))
    print(model_2.summary())

    model_cudnn = nn.CuDnnLSTMNetwork(num_cudnn_units=cudnn_units, 
                                        layer_dropout=cudnn_layer_dropout, 
                                        internal_cudnn_embedding=cudnn_embedding_dim, 
                                        internal_dropout_embedding=cudnn_dropout_embedding, 
                                        num_mlp_neurons=cudnn_mlp_units, 
                                        mlp_dropout=cudnn_mlp_dropout)
    print("{}The CUDNNLSTM Internal Model Architectural Details Are : {}".format("\n", "\n"))
    print(model_cudnn.summary())



### CuDnn-LSTM Model Training 

In [ ]:
history_cudnn = model_cudnn.fit(x_train, 
                                y_train, 
                                batch_size=batch_size if batch_size is not None else 16, 
                                epochs=num_epochs if num_epochs is not None else 20, 
                                validation_split=validation_split if validation_split is not None else 0.15, 
                                shuffle=True)

### The Stacked-LSTM Model Training

In [ ]:
history_1 = model_1.fit(x_train, 
                        y_train, 
                        batch_size=batch_size if batch_size is not None else 16, 
                        epochs=num_epochs if num_epochs is not None else 20, 
                        validation_split=validation_split if validation_split is not None else 0.15, 
                        shuffle=True)

### Compound-LSTM Model Training

In [ ]:
history_2 = model_2.fit(x_train, 
                        y_train, 
                        batch_size=batch_size if batch_size is not None else 32, 
                        epochs=num_epochs if num_epochs is not None else 75, 
                        validation_split=validation_split if validation_split is not None else 0.15, 
                        shuffle=True)

### The three plots of the model's training and validation.

In [ ]:
# summarize history for first model loss
plt.plot(history_cudnn.history['loss'])
plt.plot(history_cudnn.history['val_loss'])
plt.title('CuDnn LSTM Model Loss Graph')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

# summarize history for second model loss
plt.plot(history_1.history['loss'])
plt.plot(history_1.history['val_loss'])
plt.title('Stacked LSTM Model Loss Graph')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

# summarize history for second model loss
plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title('Compound LSTM Model Loss Graph')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

### Final Validation of the trained models on the testing dataset.

In [ ]:
predictions_cudnn = model_cudnn.predict(x_test)
predictions_cudnn = scaler.inverse_transform(predictions_cudnn)
rmse = np.sqrt(np.mean(predictions_1 - y_test) ** 2)
print("The Validation RMSE of the designed model is : {}".format(rmse))


In [ ]:
predictions_1 = model_1.predict(x_test)
predictions_1 = scaler.inverse_transform(predictions_1)
rmse = np.sqrt(np.mean(predictions_1 - y_test) ** 2)
print("The Validation RMSE of the designed model is : {}".format(rmse))


In [ ]:
predictions_2 = model_2.predict(x_test)
predictions_2 = scaler.inverse_transform(predictions_2)
rmse = np.sqrt(np.mean(predictions_2 - y_test) ** 2)
print("The Validation RMSE of the designed model is : {}".format(rmse))


In [ ]:
data = stock_data.filter(['Close'])
train = data[:training_data_len]
validation = data[training_data_len:]
validation['Predictions'] = predictions_cudnn
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(train)
plt.plot(validation[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
data = stock_data.filter(['Close'])
train = data[:training_data_len]
validation = data[training_data_len:]
validation['Predictions'] = predictions_1
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(train)
plt.plot(validation[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
data = stock_data.filter(['Close'])
train = data[:training_data_len]
validation = data[training_data_len:]
validation['Predictions'] = predictions_2
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(train)
plt.plot(validation[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()